In [1]:
import os
os.chdir('../scripts')

In [2]:
from domain_scraper import download

download.download_all()

Already Create Paths
Start url scrapping process...
178.99461793899536 seconds to scarping from domain website


In [3]:
# import required libraries
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, isnan, when, count, mean, udf, split, unix_timestamp, from_unixtime, lower
from pyspark.sql.types import StringType, IntegerType, FloatType

# init SparkSession class
spark = (
    # if available consider use yarn master node
    SparkSession.builder.master("local[*]") 
    
    # spark executor env configuration
    .config("spark.executor.memory", "8g")
    .config("spark.driver.memory", "16g")
    .config("spark.executor.cores", "2")
    .config("spark.executor.instances", "6")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    
    # jvm memory configuration
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size", "8g")
    
    # parquet file load configuration
    .config("spark.sql.repl.eagerEval.enabled", 'true')
    .config("spark.sql.parquet.cacheMetadata", 'true')
    
    # build the session
    .appName("Pyspark Start Template") # change app name here
    .getOrCreate()
)

# change default log level
spark.sparkContext.setLogLevel('ERROR')

# import libaries
import pandas as pd
import seaborn as sns
import geopandas as gdp
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import os
import geopandas as gpd
import folium
# WARNNING message is result by JVM environment setting.


from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import PCA
from statsmodels.api import families
from statsmodels.formula.api import ols, glm
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD, LassoWithSGD

from sklearn.model_selection import train_test_split

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/05 22:19:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/05 22:19:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sdf = spark.read.parquet('../data/raw/domain_data/*', hearders = True)

sdf

property_id,street,suburb,state,postcode,latitude,longitude,price,bedrooms,bathrooms,parking,property_type,is_rural,land_size,land_unit,is_retirement,url
13671355,104/300 Swanston ...,MELBOURNE,VIC,3000,-37.8106079,144.964584,$550,2,1,0,Apartment / Unit ...,false,0,m²,false,https://www.domai...
14535732,708/228 A'BECKETT...,MELBOURNE,VIC,3000,-37.8101921,144.9566,$500,2,1,0,Apartment / Unit ...,false,0,m²,false,https://www.domai...
15302782,504/350 La Trobe ...,MELBOURNE,VIC,3000,-37.81068,144.959274,$395 per week,2,1,0,Apartment / Unit ...,false,0,m²,false,https://www.domai...
15841654,285 La Trobe Street,MELBOURNE,VIC,3000,-37.8107338,144.960815,$65 per week,1,1,1,Apartment / Unit ...,false,0,m²,false,https://www.domai...
15920687,8092Y/33 Rose Lane,MELBOURNE,VIC,3000,-37.8150139,144.953873,$60 per week,1,1,1,Apartment / Unit ...,false,0,m²,false,https://www.domai...
16062720,1206/118 Russell ...,MELBOURNE,VIC,3000,-37.8135872,144.968719,$550 per week,2,1,0,Apartment / Unit ...,false,0,m²,false,https://www.domai...
16063139,625/118 Franklin ...,MELBOURNE,VIC,3000,-37.8082047,144.958908,$530,3,2,0,Apartment / Unit ...,false,0,m²,false,https://www.domai...
16063884,5401/464 Collins ...,MELBOURNE,VIC,3000,-37.8175621,144.958588,$2500 Per Week,3,3,2,Apartment / Unit ...,false,0,m²,false,https://www.domai...
16064379,1106/555 Flinders...,MELBOURNE,VIC,3000,-37.8209724,144.956482,$370 per week,1,1,0,Apartment / Unit ...,false,0,m²,false,https://www.domai...
16065974,1211/260 Spencer ...,MELBOURNE,VIC,3000,-37.8145638,144.952286,$700,2,2,0,Apartment / Unit ...,false,0,m²,false,https://www.domai...


In [5]:
sdf.count()

15439